# **Using LLM assitance**

If we want a lightweigth model that can generate Shakespearian sonnets, we realized that the model by itself is challenging. While it manages to get the goals we had set up for the train and test set, its ability to create generalized structure is limited.

However, this open an opportunity to incorporate the thing that everyone is talking about (but without just asking it to solve a task for us and trusting the black box): LLMs!

## How can we do it?

Instead of asking the LLM to generate a sonnet, we can provide it with the generated sonnet from our model and ask it to correct it. Eliminate duplicate words, strange structures, and assist line breaking. In this way, we can be more in control of the LLM, asking it to not alter our original sonnet, but do what it does best: work with patterns. Because, in the end of the day, a model like Gemini probably already "read" all sonnets written by humanity...

In [10]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
import torch

from model import TransformerModel
from utils import generate_sonnet_sampling, clean_generated_text

import sentencepiece as spm

load_dotenv()

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

Using device: cuda


In [12]:
model = TransformerModel(
    vocab_size=4000, 
    seq_len=256, 
    embedding_dim=256, 
    num_heads=8, 
    num_layers=6, 
    dropout=0.1
).to(device)

model_path = 'model/checkpoint_epoch_200_valloss_1.9365.pth'

# Load
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
    print(f"Model loaded from {model_path}")

Model loaded from model/checkpoint_epoch_200_valloss_1.9365.pth


/tmp/ipykernel_21556/2830344480.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location='cpu'))


The distribution that yielded some of the better results while reading was using sampling.

In [14]:
sp = spm.SentencePieceProcessor(model_file='tokenizer/my_tokenizer.model')

In [15]:
model.eval()

start_text = "Thy"
raw = generate_sonnet_sampling(model, sp, start_text, max_length=256, device='cuda')
generated_sonnet = clean_generated_text(raw, pad_token='<PAD>')
print(generated_sonnet)

Thy their known ruthless lament their their these muse reveal me vain imposed history's of sightcauldronion from thee. figure first within the these unwritten my of Decoright from the absent, did speak toos. 
. akatch use is from the upon myju fall their violet I baseamand reason light of monarchs bold to fall away lies wanting ardor gulf naught 
 Nor Marsing and bright Pardoning bow de sight lent. 
 Thus on Death bow to nature call, these lines, gracing her head richness 
 For grow from those pe Ardor of grasp of hollow glory, adorned confess my feeble they have victor fate my far from their guest, claim I think beneath the sun'll declare; qu 
 Here song amidst grace flares gates. 
 Why fearch thrives, seek treasure voy and remorselessiness die of stars lost, Shall fade, yearning totholo doessers, reviled.


## Creating a prompt

In [28]:
# Start the use of the API
genai.configure(api_key=GEMINI_API_KEY)

# Make our prompt here
prompt = f"""
You are an expert and a teacher in Shakespearing sonnets. Your goal is to take your student's sonnet and help him polish it. You need to pay attention to:
* Duplicate words
* Repetitive phrases
* Line breaks
* Slight mispells
* Words that were clearly stuck together

You are NOT allowed to change the student's words or their meaning, that will make him sad!

Your output should be the new sonnet. No explanations, no anything more or less.

Here is the student's sonnet:
{generated_sonnet}
"""

generation_config = genai.GenerationConfig(
    max_output_tokens=256,
    temperature=1.0,
)

# Use our prompt 
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

response = model.generate_content(prompt,
                                  generation_config=generation_config)

print("Response from Gemini:")
print()
print(response.candidates[0].content.parts[0].text)

Response from Gemini:

Thy known ruthless lament, these muse reveal, vain, imposed history's sight, a cauldron from thee.
Figure first within unwritten Decoright, from the absent, did speak.
A catch, use is from upon my fall, their violet, I base my reason; light of monarchs bold, to fall away, lies wanting ardor, a gulf naught.
Nor Mars' bright pardoning bow, sight lent.
Thus, on Death's bow to nature's call, these lines, gracing her head's richness,
grow from those; ardor's grasp of hollow glory, adorned, confess my feeble; they have victor fate, my far from their guest, claim I think beneath the sun declare.
Here song, amidst grace, flares gates.
Why search thrives, seek treasure; voyaging remorselessness, die of stars lost, shall fade, yearning.



In [26]:
print(generated_sonnet)

Thy their known ruthless lament their their these muse reveal me vain imposed history's of sightcauldronion from thee. figure first within the these unwritten my of Decoright from the absent, did speak toos. 
. akatch use is from the upon myju fall their violet I baseamand reason light of monarchs bold to fall away lies wanting ardor gulf naught 
 Nor Marsing and bright Pardoning bow de sight lent. 
 Thus on Death bow to nature call, these lines, gracing her head richness 
 For grow from those pe Ardor of grasp of hollow glory, adorned confess my feeble they have victor fate my far from their guest, claim I think beneath the sun'll declare; qu 
 Here song amidst grace flares gates. 
 Why fearch thrives, seek treasure voy and remorselessiness die of stars lost, Shall fade, yearning totholo doessers, reviled.
